In [ ]:
urban_dir = "data_urbanisme"
parcels_file = "parcelles_filtrées.geojson"

# 📦 Charger les parcelles filtrées
filtered_parcels = gpd.read_file(parcels_file)
if filtered_parcels.crs != "EPSG:2154":
    filtered_parcels = filtered_parcels.to_crs("EPSG:2154")
filtered_parcels = filtered_parcels[filtered_parcels.geometry.is_valid]

urban_chunks = []

for filename in tqdm(os.listdir(urban_dir), desc="Départements zones urbanisme"):
    if filename.endswith(".parquet"):
        code_dept = filename.split("-")[1].split(".")[0]
        urban_path = os.path.join(urban_dir, filename)

        urban_gdf = gpd.read_parquet(urban_path)
        if urban_gdf.crs != "EPSG:2154":
            urban_gdf = urban_gdf.to_crs("EPSG:2154")
        urban_gdf = urban_gdf[urban_gdf.geometry.is_valid]

        # 🔍 Intersections réelles avec overlay
        intersected = gpd.overlay(filtered_parcels, urban_gdf[['geometry', 'typezone']], how="intersection")

        # 📏 Calculer la surface de l'intersection
        intersected["surface_intersection"] = intersected.geometry.area

        # 🧼 Ne garder que les zones qui couvrent > 10 m² (à adapter si besoin)
        intersected = intersected[intersected["surface_intersection"] > 10]

        if not intersected.empty:
            intersected["departement"] = code_dept
            urban_chunks.append(intersected)

# 📚 Fusion
all_urban_on_parcels = pd.concat(urban_chunks, ignore_index=True)

# 💾 Export
all_urban_on_parcels.to_file("zones_urbanisme_sur_parcelles.geojson", driver="GeoJSON")
print(f"✅ {len(all_urban_on_parcels)} zones d’urbanisme extraites et exportées.")
